<a href="https://www.kaggle.com/code/gabripo93/the-perfect-match-for-your-tech-and-business-needs?scriptVersionId=209720989" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# In-chat Multiagents to Find the Right Company and Generate Clause-by-Clause Reports for Tenders 📑💼

The following Kaggle notebook takes advantage of Gemini's long context window to achieve the following objectives:

- Analyze technical and commercial tenders for a project. 📊
- Assess the compatibility of companies' products and solutions with tender documents. 🔍
- Identify the best company and product-service combination to execute the project, generating a clause-by-clause report with compliant and non-compliant specifications. 📋✅❌

## Notebook Structure 📓

The notebook is divided into different sections, each with a specific objective:

- Dataset load (see *Relevant Project and Open points* chapter for data generation)
- Tenders for a project are parsed, converting their information into text. 📝
- Information scraped from various companies' websites is loaded as text. 🕸️
- All text is processed by Gemini using different prompts, combining multi-agent reasoning, chain of thoughts, and in-chat memory. 🤖💭

## Multi-agent Reasoning 🧠

Multi-agent reasoning is implemented by segmenting tasks and delegating responsibilities to distinct roles. For example:

- **Technical and Commercial Tender Agents**: Separate prompts (tender_prompt_template_technical and tender_prompt_template_commercial) guide the roles of the technical tender engineer and commercial tender manager. Each agent has distinct objectives: identifying and summarizing technical or commercial requirements within tenders. This multi-agent structure ensures detailed and domain-specific analyses. 👷‍♂️💼

- A distinct prompt is also prepared for analyzing companies (e.g., SIEMENS and HITACHI) to match tender requirements with their products and solutions (get_response_companies_info). This allows tailored reasoning for comparing affinities between tenders and company offerings. 🏢🔄

## Chain of Thoughts 🧩

The chain of thoughts approach decomposes complex tasks into sequential, step-by-step actions, ensuring methodical problem-solving. In both technical and commercial prompts, we used phrases like "Think step by step" to guide the agent toward incremental reasoning. This ensures that requirements are dissected and analyzed in detail. 🔍🧠

The user prompt specifies a structured approach to calculating an affinity score, prompting the agent to explicitly explain the calculation process. Finally, in the Clause-by-Clause Analysis, the final prompt directs the agent to meticulously compare tender requirements with company specifications, maintaining a clear progression in thought. This approach is embedded in the tender query processing and the affinity scoring logic in user_prompt_match and final_prompt, encouraging logical progression in the analysis. 📈🔗

## In-chat Memory 🗃️

The code uses in-chat memory to maintain conversational context across multiple interactions.

In-chat memory stores all the interactions from the technical and commercial tender analysis, keeping track of the responses from the different roles (e.g., technical engineer, commercial manager, sales manager). This memory allows to build upon the context of earlier prompts without having to constantly reprocess the same information. With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.

This functionality is facilitated by:

- **Chat History Preservation**: The function add_history_to_chat appends user queries and model responses (e.g., for tenders or company analyses) to history_chat. This ensures continuity, enabling the model to refer back to previous inputs and outputs during subsequent exchanges. 📝🔄

- Prompts such as system_prompt and user_prompt leverage the accumulated chat history to enhance the depth and relevance of responses. For example, when computing affinity scores or performing a clause-by-clause analysis, the model can reference earlier content in the chat_with_memory object. This allows continuous improvement of the prompt and on the information stored in the chat. 🗣️🔍

## Conclusion for the Use Case 🤔

Using a long context window instead of Retrieval-Augmented Generation (RAG) for this use case was particularly beneficial due to the task's nature, which involves reasoning across interdependent documents while maintaining conversational continuity and ensuring consistent context for decision-making. The unified context allows the model to cross-reference tender requirements and company offerings directly, ensuring cohesive and accurate analysis. This is particularly advantageous for tasks like affinity scoring, which require simultaneous consideration of multiple data points. 📊🔗

The notebook's approach scales better for handling multiple queries simultaneously, as it avoids the bottleneck of sequential agent calls. For new tender projects, it's only necessary to update the in-chat memory and add new prompts for adding new in-chat agents. 🔄🔄

In summary, why did we decide to build this notebook?

1. **Holistic Context Retention** 📚: By storing the entire history of tender analyses (both technical and commercial) and company product evaluations, the model retains a comprehensive understanding of all previously provided information. This holistic context allows the model to reason about how specific requirements and offerings interrelate across multiple prompts. In RAG, the system retrieves only the most relevant chunks of information for each query; this efficient approach can lead to fragmented analyses, potentially overlooking interconnections.

2. **Interdependent Analysis** 🔄: This task involves comparing multiple tenders against products and solutions offered by different companies, followed by calculating an affinity score and conducting a clause-by-clause compliance analysis. These steps require accessible and integrated information from previous steps. RAG typically retrieves context independently for each query, which might result in a loss of nuance or context-dependent reasoning, especially when relationships between multiple documents must be preserved. A long context window ensures the model has immediate access to the entire conversational flow and insights developed so far.

3. **Dynamic Multi-Agent Collaboration** 🤝: By maintaining a long context, the system can simulate multi-agent collaboration, allowing outputs from technical engineers, commercial managers, and sales managers to flow into a unified reasoning framework. In RAG, each role’s analysis would require re-retrieving relevant information, possibly leading to inconsistencies or duplications. A long context window naturally informs each role, creating a seamless chain of thought.

4. **Reduced Query Overhead** 🔄: Long context windows reduce the need for multiple retrieval calls, making the process more efficient in scenarios where information is revisited or refined iteratively. RAG introduces latency and computational costs because each query requires searching and ranking document chunks. A long context window allows for continuous focus on the task, with all prior exchanges readily available.

5. **Affinity Score Calculation** 📈: Computing an affinity score across companies for tenders requires integrating technical and commercial analysis alongside company data. This step benefits significantly from the model's ability to access all previous responses simultaneously. In RAG, affinity scoring would require separate retrievals of technical requirements, commercial requirements, and company data for each tender. This could introduce discrepancies if context for one query is inadvertently excluded during retrieval.

6. **Clause-by-Clause Compliance Analysis** 🔄: Clause-by-clause analysis relies on cross-referencing previously extracted requirements with company offerings. The long context window allows the model to directly reference earlier inputs and outputs without reloading or retrieving. RAG retrievals for clause-by-clause analysis might lead to inconsistencies if prior reasoning is split across multiple retrievals. A long context window ensures the model "remembers" and applies earlier analyses cohesively.

### Related Projects and Open Points 📁

The data generation and cleaning is performed with another repo stored in github:  https://github.com/gabripo/kaggle-gemini-long-context.

In the past few months, we also implemented a multi-agent framework (LumadaAI) using LangChain and OpenAI, where each company was represented by a dedicated agent. **LumadaAI** is publicly available at https://github.com/SecchiAlessandro/LumadaAI. This framework featured a supervisor agent that dynamically routed user queries to the most relevant company-specific agent based on the query context. While innovative, this approach faced challenges in stability, accuracy, and efficiency, making the current solution more effective. As agents operated independently, generating combined solutions from different companies was difficult. Additionally, for each query, the supervisor needed to perform additional reasoning before invoking an agent. If a query was relevant to multiple agents, the framework had to perform sequential calls, compounding latency. The current solution with centralized reasoning ensures consistent application of logic and context. By avoiding the intermediate step of agent selection, it directly processes queries with unified context, reducing latency significantly. 

**EasyRAG** (https://github.com/gabripo/easyrag) is another RAG tool that performs RAG over locally stored documents. We are benchmarking this tool with Gemini's long context window: adding one or more PDFs to Gemini's context window could provide more precise insights than the RAG approach. 📈

## Conclusion 🔽

The centralized, long context window approach provides clear advantages in stability, response time, and accuracy over the earlier multi-agent framework. It highlights the importance of selecting a system architecture that aligns with the specific demands of the use case, particularly for complex, multi-faceted analyses like those in tender evaluations, clause-by-clause generation, and company affinity scoring. 📊🔗

#### The long context window acts as a shared workspace, recording and making all agent outputs accessible for seamless and holistic reasoning. In today's interconnected world, where partnerships and synergies are essential to addressing complex challenges, we envision a tool that enables continuous reasoning, uncovers new patterns and solutions, and minimizes the fragmentation of insights. 🌐🔍💡



In [1]:
# import Python libraries
import os
import json
from IPython.display import Markdown

In [2]:
# auxiliary function to read JSON files
def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

In [3]:
# auxiliary Python decorator to execute a function again, if its execution fails
# this is helpful when calling the Gemini's API since Gemini has a rate limiter and, if an execution fails for that, there will be some waiting time before retrying
import time

def retry_on_failure(wait_time_seconds=60, max_retries=5):
    def decorator_retry(func):
        
        def wrapper_retry(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries < max_retries:
                        print(
                            f"Function failed with error: {e}. Retrying in {wait_time_seconds} seconds... (Attempt {retries}/{max_retries})"
                        )
                        time.sleep(wait_time_seconds)
                    else:
                        print(f"Function failed after {max_retries} attempts.")
                        raise e
        return wrapper_retry

    return decorator_retry

In [4]:
dataset_path = '/kaggle/input/tenders-and-companies-websites'
working_path = '/kaggle/working'

In [5]:
!mkdir -p /kaggle/working/tenders
tenders_working_path = os.path.join(working_path, 'tenders')

!mkdir -p /kaggle/working/companies
companies_working_path = os.path.join(working_path, 'companies')

# Build a chat with Gemini

In [6]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


In [7]:
# the decorator ensures that, if an error occurs, the function will be executed again
@retry_on_failure(wait_time_seconds=60, max_retries=3)
def ask_gemini(prompt, chat_with_memory=None, model=None, history=[]):
    """
    function to call Gemini, providing chat history
    if a chat is already available, it will be used
    """
    if chat_with_memory == None:
        # since no chat is already available, create a new one
        chat_with_memory = model.start_chat(history=history)
    
    response = chat_with_memory.send_message(prompt)
    return response, chat_with_memory

# Analyze the tenders

In [8]:
# read the json file related to tenders from the input dataset
tenders_info_json_path = os.path.join(dataset_path, 'tenders_info.json')
tenders_info = read_json_info(tenders_info_json_path)

# tenders_info is a dictionary, where the key is the name of the tender file and the related value its information
# print(tenders_info["tender_wind.pdf"])

In [9]:
# list the processed tender files
tenders = tenders_info.keys()

In [10]:
tender_prompt_template_technical = """
You are an experienced technical tender engineer. 
The document you have is a tender, that contains also technical requirements for a project.
Think step by step on how to look for the relevant technical requirements and make a detailed summary.
The content of the document is: """
tender_prompts_technical = []
for info in tenders_info.values():
    tender_prompts_technical.append(f"You have a document called {info['name']} . " + tender_prompt_template_technical + f"{info['content']}")

In [11]:
tender_prompt_template_commercial = """
You are an experienced commercial tender manager. 
The document you have is a tender, that contains also commercial requirements for a project.
Think step by step on how to look for the relevant commercial requirements and make a detailed summary.
The content of the document is: "
"""
tender_prompts_commercial = []
for info in tenders_info.values():
    tender_prompts_commercial.append(f"You have a document called {info['name']} . " + tender_prompt_template_commercial + f"{info['content']}")

In [12]:
@retry_on_failure(wait_time_seconds=60)
def get_responses_tenders(subject, tender_prompts):
    tenders_json_file_path = os.path.join(tenders_working_path, f'tenders_{subject}.json')
    
    if os.path.exists(tenders_json_file_path):
        responses = read_json_info(tenders_json_file_path)
        print(f"tender_{subject}: Responses loaded from file {tenders_json_file_path}")
    else:
        for tender_prompt, tender_name in zip(tender_prompts, tenders):
            print(f"tender_{subject}: Generating response for tender {tender_name} ...")
            response, _ = ask_gemini(prompt = tender_prompt, model = model)
            #print(response.text)

            responses = {}
            responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
            print(f"tender_{subject}: Response for tender {tender_name} generated.")
    
        with open(tenders_json_file_path, 'w') as f:
            json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"tender_{subject}: Responses stored into {tenders_json_file_path}")
    
    print(f"tender_{subject}: Analysis concluded!\n")
    return responses

# each call of get_responses_tenders() will generate a tenders_{subject}.json file
# each generated file so will contain the Gemnini's responses for a given subject
response_technical = get_responses_tenders("technical", tender_prompts_technical)
response_commercial = get_responses_tenders("commercial", tender_prompts_commercial)

tender_technical: Responses loaded from file /kaggle/working/tenders/tenders_technical.json
tender_technical: Analysis concluded!

tender_commercial: Responses loaded from file /kaggle/working/tenders/tenders_commercial.json
tender_commercial: Analysis concluded!



# Analyze the companies products and solutions

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.

The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> The generation of information can be found in the Kaggle Notebook https://github.com/gabripo/kaggle-gemini-long-context.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

In [13]:
companies_info_json_path = os.path.join(dataset_path, 'companies_info.json')
companies_info = read_json_info(companies_info_json_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

In [14]:
@retry_on_failure(wait_time_seconds=60)
def get_response_companies(company_name):
    companies_json_file_path = os.path.join(companies_working_path, f'companies_{company_name}.json')
    
    if os.path.exists(companies_json_file_path):
        responses = read_json_info(companies_json_file_path)
        print(f"companies_{company_name}: Responses loaded from file {companies_json_file_path}")
    else:
        print(f"companies_{company_name}: Generating response for company {company_name} ...")
        responses = {}
        company_prompt = f"These are the information of products and solutions for the company {company_name} : {companies_info[company_name]}"
        response, _ = ask_gemini(prompt = company_prompt, model = model)
        responses[company_name] = {'prompt': company_prompt, 'answer': response.text}

        with open(companies_json_file_path, 'w') as f:
                json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"companies_{company_name}: Responses stored into {companies_json_file_path}")

    print(f"companies_{company_name}: Response for company {company_name} generated!")
    return responses

# each call of get_responses_companies() will generate a companies_{company_name}.json file
# each generated file so will contain the Gemnini's responses for a given company
# the purpose of generating responses given companies information is to store it in the chat history


In [15]:
response_hitachi = get_response_companies("HITACHI")

companies_HITACHI: Responses loaded from file /kaggle/working/companies/companies_HITACHI.json
companies_HITACHI: Response for company HITACHI generated!


In [16]:
response_siemens = get_response_companies("SIEMENS")

companies_SIEMENS: Responses loaded from file /kaggle/working/companies/companies_SIEMENS.json
companies_SIEMENS: Response for company SIEMENS generated!


# Build a chat history based on previous prompts

In [17]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []

def add_history_to_chat_single(response, user, history_chat):
    query = Part()
    query.text = f"{user}: {response['prompt']}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))
    return

def add_history_to_chat(responses, user, history_chat):
    for response in responses.values():
        add_history_to_chat_single(response, user, history_chat)
    return 

add_history_to_chat(response_technical, "technical engineer", history_chat)
add_history_to_chat(response_commercial, "commercial manager", history_chat)
add_history_to_chat(response_siemens, "sales manager for siemens", history_chat)
add_history_to_chat(response_hitachi, "sales manager for hitachi", history_chat)

## Test the chat history

In [18]:
prompt_roles = "Which are the roles given in the prompt from the user? There are only two for tenders and one for company"
response_roles, gemini_chat = ask_gemini(prompt=prompt_roles, model=model, history=history_chat)

Markdown(response_roles.text)

The prompt mentions three roles:

1. **Technical Engineer:** This role focuses on evaluating the technical specifications and requirements within a tender document.

2. **Commercial Manager:** This role centers on analyzing the commercial aspects and requirements within a tender document, including pricing, warranties, and service agreements.

3. **Sales Manager (for Siemens and Hitachi):**  This role involves understanding a company's product portfolio and capabilities to determine which products and services are relevant to respond to a tender.  It also involves understanding the market and presenting the products in a way to win the tender.


In [19]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_roles, 'answer': response_roles.text}, "technical engineer", history_chat)

# Find the most suitable company

In [20]:
prompt_match = """

1. For company SIEMENS and HITACHI, find the respective relevant products and solutions with respect to the analyzed tenders.
   The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1. Explain the way how you computed this percentage.

"""

In [21]:
print("Finding the most suitable company for the tenders ...")
response_match, gemini_chat = ask_gemini(prompt=prompt_match, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_match.text)

Finding the most suitable company for the tenders ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/3)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 2/3)
Response to the prompts is ready!


## 1. Relevant Siemens and Hitachi Products & Solutions for the Barclayville Solar Power Plant Tender

This analysis identifies Siemens and Hitachi products relevant to the Barclayville Solar Power Plant tender based solely on the provided text.  The tender's commercial and technical requirements are broad, allowing for multiple interpretations and vendor selections.

**Siemens:**

The tender's scope is significantly broader than Siemens' typical offerings; Siemens primarily focuses on large-scale power generation and transmission.  However, several Siemens products are potentially relevant:

* **Omnivise Asset Management:**  This software suite directly addresses the tender's need for a monitoring system (item 9) and aligns with the overall digitalization aspects of the project.  The modular nature allows for tailoring to this specific project.

* **SGT-800 or SGT-400 Gas Turbines (Item 5):** These mid-size industrial gas turbines could fulfil the 180 kW (225 kVA) diesel genset requirement, although a smaller unit might be more appropriate.  Siemens' service offerings (long-term programs, overhauls, spare parts) are also highly relevant.

* **SGen Generators (Item 5, 20, 21):** Siemens offers generators that may fit requirements for the diesel genset and customer connections.  Specific model selection depends on the required capacity.

* **High-voltage substations:** Siemens offers various high-voltage substation solutions (item 12) including the civil works, which may be a suitable response to item 12.

* **High-Voltage Refurbishment Solutions:**  Relevant if components from other vendors require upgrading or maintenance.


**Hitachi Energy:**

Hitachi Energy's product portfolio aligns more directly with many of the tender requirements, especially for the mini-grid components:

* **nMarket:** This software addresses the tender's commercial aspects by offering tools for energy trading and risk management, particularly relevant given the potential to sell surplus power.

* **TXpert™ Enabled distribution transformers (Items 17, 18, 19):**  These address the requirements for step-down transformers, with digital monitoring capabilities adding value.

* **Power Quality Filters (Item 6):**  Hitachi Energy’s active filters would address the tender's requirement for minimizing power loss and could be a value-add in the electrical BOS.

* **RelCare:**  This asset management platform directly supports the tender's requirement for long-term maintenance (item 25) and spare parts management (item 23).  It could address the warranty requirements (item 10) as well.

* **Modular Switchgear Monitoring (MSM):**  For monitoring critical parameters in the substation and other equipment.

* **Skid-mounted Substations (Item 12):** Hitachi Energy's pre-fabricated substation solutions could offer a faster, more efficient approach.


## 2. Affinity Score Calculation

The affinity score calculation is subjective and based on a qualitative assessment of how well each company's offerings meet the tender's requirements.  A more precise calculation would require detailed pricing and technical specifications not available in the provided text.

**Methodology:**

1. **Requirement Weighting:**  Assign weights to each item in the tender based on its perceived importance.  For simplicity, let's assume equal weighting for all 25 items (4% each).

2. **Company Alignment:** For each item, assess how well Siemens and Hitachi's offerings address it.  Assign scores from 0 (no alignment) to 1 (perfect alignment). Fractional scores are acceptable to reflect partial alignment.


3. **Weighted Score:** Multiply each item's weight (4%) by the alignment score.

4. **Total Weighted Score:** Sum the weighted scores for each company.

5. **Affinity Percentage:** The total weighted score represents the affinity percentage.

**Example (Illustrative – Actual scores would require more detailed tender information):**

Let's suppose a simplified assessment (due to incomplete tender details):

| Item | Weight | Siemens Alignment | Siemens Weighted Score | Hitachi Alignment | Hitachi Weighted Score |
|---|---|---|---|---|---|
| 1-9 (Technical Spec) | 36% | 0.3 | 10.8% | 0.7 | 25.2% |
| 10-25 (Commercial & Service) | 64% | 0.4 | 25.6% | 0.8 | 51.2% |
| **Total** | **100%** |  | **36.4%** |  | **76.4%** |

**Affinity Scores:**

* **Siemens:** 36.4% affinity
* **Hitachi Energy:** 76.4% affinity

**Conclusion:**

Based on this illustrative example, Hitachi Energy shows a much stronger affinity to the tender's requirements than Siemens.  This is primarily due to the broader applicability of Hitachi's portfolio to the mini-grid aspects of the project and the inclusion of relevant software solutions.  However, this is a *simplified* example. A real-world analysis would need far more detail on the specific requirements of each item, including pricing and technical specifications.  The weighting of requirements would also significantly influence the results.


In [22]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_match, 'answer': response_match.text}, "technical engineer", history_chat)

# Generate the clause-by-clause

In [23]:
user_prompt = """

Consider the company with the highest affinity score 
and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements
of the tender with respect to the selected company. Report also the URL of the source where you found the informations.

"""

In [24]:
system_prompt = """
You are an experienced team of business development managers and tender engineers, commercial managers.
You need to create a detailed clause by clause from the tender documentations and the most affine company specifications.
"""

In [25]:
print("Generating the clause by clause ...")
prompt_clause_by_clause = f"{system_prompt} {user_prompt}"
response_clause_by_clause, gemini_chat = ask_gemini(prompt=prompt_clause_by_clause, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_clause_by_clause.text)

Generating the clause by clause ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/3)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 2/3)
Response to the prompts is ready!


Based on the previous analysis, Hitachi Energy demonstrates a higher affinity score for the Barclayville Solar Power Plant tender.  Therefore, this clause-by-clause analysis focuses on Hitachi Energy's capabilities compared to the tender requirements.  Note that this analysis relies solely on the information provided in the previous prompt; complete product specifications and precise pricing are not available, making definitive compliance assessments impossible.  Therefore, this analysis is illustrative and should be considered a high-level preliminary assessment.

**Tender Document Clause-by-Clause Analysis (Hitachi Energy)**

The tender document is organized into numbered items (1-25)  with detailed specifications provided for some.  The following analysis follows that structure, indicating Hitachi Energy's alignment with both the technical and commercial implications of each clause.  URLs are not provided since the information is derived from the previously supplied text.

**Note:**  "Compliant" indicates a strong likelihood of Hitachi Energy meeting the requirement based on their stated capabilities. "Potentially Compliant" signifies that more information is needed to confirm full compliance.  "Not Compliant" means that Hitachi's stated capabilities do not directly address the requirement.  "Commercial Implications" highlight the financial and contractual aspects.


| Item No. | Tender Clause Description                                                                          | Hitachi Energy Alignment | Technical Compliance | Commercial Implications                                                                                                                                           |
|---|---|---|---|---|
| 1 | Solar PV Array (200 kWp capacity, >80% power output after 25 years, IEC/UL certified)                         | Not Compliant             | Not Compliant       | Hitachi Energy does not directly supply solar PV arrays. This would require subcontracting, impacting project management and potentially increasing costs.                                |
| 2 | PV Mounting Structure (non-corrosive, adjustable tilt, wind rating 150 kph)                               | Potentially Compliant     | Potentially Compliant | Hitachi Energy doesn't explicitly mention PV mounting structures.  They supply transformer components, suggesting they might have suitable materials and could likely source compliant structures.  Subcontracting remains a possibility. |
| 3 | Inverter (140 kW nominal capacity, >95% efficiency, protection features, communication protocols)       | Potentially Compliant     | Potentially Compliant | Hitachi Energy doesn't list specific inverters, but their involvement in power quality solutions suggests they could likely procure a compliant inverter.  Subcontracting is possible.                                 |
| 4 | Battery (400 kWh capacity, Lithium-ion, cycle life, self-discharge rate, warranty)                          | Potentially Compliant     | Potentially Compliant | Hitachi Energy offers BESS (Battery Energy Storage Systems), including BlueVault™ and SIESTART.  The specific battery technology and performance would need verification against the tender specifications.  Warranty terms are key. |
| 5 | Diesel Genset (180 kW, cold starting, control system, protection features)                               | Not Compliant             | Not Compliant       |  Hitachi Energy’s core business does not encompass diesel gensets.  This item would require subcontracting.                                                                       |
| 6 | Electrical BOS (cable specifications, junction box requirements)                                         | Potentially Compliant     | Potentially Compliant | Hitachi Energy supplies various electrical components (transformers, switchgear, etc.), and their expertise in power systems makes them capable of providing compliant cables and junction boxes. Subcontracting might be needed for specific items. |
| 7 | Powerhouse buildings and parking                                                                      | Not Compliant             | Not Compliant       |  Hitachi Energy does not offer building construction services.  This would need to be subcontracted.                                                                      |
| 8 | Installation labor, tools, and equipment                                                               | Potentially Compliant     | Potentially Compliant | Hitachi Energy offers installation and commissioning services.  The specific skills and equipment for this project need confirmation.  Subcontracting is a possibility for specialized tasks. |
| 9 | Monitoring System (SCADA, parameters to be monitored, remote access)                                   | Compliant                | Compliant            | Hitachi Energy's Omnivise Asset Management software could fulfill this requirement.  Their expertise in digital solutions suggests they can tailor a system to meet the specified parameters and provide remote access.    |
| 10| Warranty (minimum 2 years for main system, 80% capacity retention for battery after 2 years)             | Compliant                | Compliant            | Hitachi Energy provides warranties.  The specific terms and conditions would need to be aligned with the tender.                                                               |
| 11| Manuals for installation, maintenance, and troubleshooting                                              | Potentially Compliant     | Potentially Compliant |  Hitachi Energy provides training and documentation. The specific format and content of the manuals require confirmation.                                                        |
| 12| Sub-station (0.4/11 kV, 500 kVA) including civil works                                                 | Compliant                | Potentially Compliant | Hitachi Energy provides substation solutions including various switchgear and transformer options.  Civil works would need subcontracting.                                          |
| 13| Mini-grid Survey, Design, Mobilization Prelims                                                          | Potentially Compliant     | Potentially Compliant | Hitachi Energy's expertise in grid design and construction suggests they could handle the survey and design elements.  Mobilization and preliminary work would likely be subcontracted.                         |
| 14-22| Mini-grid infrastructure (cables, transformers, customer connections, SHS)                             | Compliant                | Potentially Compliant | Hitachi Energy's extensive portfolio of transformers, switchgear, and related equipment makes them a likely candidate to provide compliant solutions.  Specific model selection is crucial.                 |
| 23 | Spare parts (list, quantities, sources, prices, after-sales service)                                   | Compliant                | Compliant            | RelCare asset management system directly supports spare parts management. The detailed list and after-sales service would need to be provided.                                         |
| 24 | Training                                                                                             | Compliant                | Compliant            | Hitachi Energy offers training programs. The specific content and number of trainees would need to be clarified.                                                              |
| 25 | After-sales services (3 years)                                                                      | Compliant                | Compliant            | Hitachi Energy's service offerings align with the three-year after-sales service requirement.  Specific terms and conditions must be defined.                                           |


**Overall Assessment:**

Hitachi Energy's comprehensive portfolio of products and services, coupled with its digital solutions (nMarket, RelCare, TXpert™), positions them favorably for this tender. However, certain elements (items 1, 5, 7) require subcontracting, which needs careful consideration from a project management and cost perspective.  Detailed technical specifications and warranty terms must be meticulously compared to the tender requirements to determine precise compliance. The commercial aspects, including pricing and payment terms, are critical to overall bid competitiveness.


In [26]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_clause_by_clause, 'answer': response_clause_by_clause.text}, "technical engineer", history_chat)

## Count the overall tokens

The total number of token can be computed by counting the tokens of history_chat, since the new responses have been appended to it for each call of Gemini.

In [27]:
print(f"{model.count_tokens(history_chat)=}")

model.count_tokens(history_chat)=total_tokens: 667082



# Alternative usage of Gemini: Context caching

As the entire dataset consists in JSON files, it could be cached using the Context caching functionality of Gemini:

https://ai.google.dev/gemini-api/docs/caching?lang=python

With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.

In [28]:
# to ensure that no caching limit is exceeded, flush all the already available caches
from google.generativeai import caching

def delete_caches() -> None:
    for c in caching.CachedContent.list():
        print(f"Deleting cache named \"{c.display_name}\" ...")
        c.delete()
    print("All the caches have been deleted!")

delete_caches()

All the caches have been deleted!


In [29]:
import google.generativeai as genai
import time

@retry_on_failure(wait_time_seconds=10, max_retries=2)
def process_file_for_caching(file_path: str) -> genai.types.file_types.File | None:
    if os.path.exists(file_path):
        loaded_file = genai.upload_file(file_path)
        
        while loaded_file.state.name == "PROCESSING":
            print(f"Processing file {file_path} ...")
            time.sleep(2)
            loaded_file = genai.get_file(loaded_file.name)
        print(f"Processing file {file_path} completed. Available at {loaded_file.uri}")
        
        return loaded_file
    else:
        return None

# the process_file_for_caching() function will make the input file as available for caching
tenders_info_file = process_file_for_caching(tenders_info_json_path)
companies_info_file = process_file_for_caching(companies_info_json_path)

Processing file /kaggle/input/tenders-and-companies-websites/tenders_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/ruf5qroaugxl
Processing file /kaggle/input/tenders-and-companies-websites/companies_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/tmzr5uvnjvn


In [30]:
cache_instructions = f"""
{system_prompt}
The information you need is in the JSON files you have access to.
"""

In [31]:
import datetime

@retry_on_failure(wait_time_seconds=60, max_retries=2)
def build_cache_from_contents(model_name: str ='gemini-1.5-flash-002', cache_name: str='cache', instructions: str="Use the information to answer", resources: list=[], minutes_available: int=10):
    cache = caching.CachedContent.create(
        model=model_name,
        display_name=cache_name, # used to identify the cache
        system_instruction=(instructions),
        contents=resources,
        ttl=datetime.timedelta(minutes=minutes_available),
    )
    
    model_with_cache = genai.GenerativeModel.from_cached_content(cached_content=cache)
    return model_with_cache

files_to_cache = [tenders_info_file, companies_info_file]
model_with_cache = build_cache_from_contents(cache_name='tenders and companies info', instructions=cache_instructions, resources=files_to_cache)

In [32]:
def list_available_caches():
    cache_list = list(caching.CachedContent.list())
    if len(cache_list) == 0:
        print("Empty cache!")
        return []
        
    for c in cache_list:
        print(f"Available cache with name \"{c.display_name}\"\n    model: {c.model}\n    created: {c.create_time}\n    expires: {c.expire_time}\n    tokens: {c.usage_metadata.total_token_count}")
        # print(c)
    return cache_list

caches = list_available_caches()

Available cache with name "tenders and companies info"
    model: models/gemini-1.5-flash-002
    created: 2024-11-26 11:30:57.645119+00:00
    expires: 2024-11-26 11:40:57.223686+00:00
    tokens: 668315


## Test if the cache works

In [33]:
def ask_gemini_with_cache_dummy_response(model, prompt: str=""):
    """
    Function to handle a possible error by Gemin when generating a response
    """
    try:
        response = model.generate_content(prompt)
    except Exception as exc:
        print(f"Execution in model with caching failed: {exc}")
        print("An empty response will be returned")
        
        class DummyResponse:
            def __init__(self):
                self.text = "MODEL WITH CACHING FAILED TO GENERATE RESPONSE"
        response = DummyResponse()
        
    return response

@retry_on_failure(wait_time_seconds=1, max_retries=2)
def ask_gemini_with_cache(model, prompt: str=""):
    """
    Wrapper for a Gemini model that uses cache
    """
    response = model.generate_content(prompt)
    return response

In [34]:
response_cached_files = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt="Describe the documents you have access to")

Markdown(response_cached_files.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Generate the clause-by-clause with cached content

In [35]:
response_cached_clause_by_clause = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt=f"Use the documents you have access to, to answer.\n {user_prompt}")

Markdown(response_cached_clause_by_clause.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Deleting the generated caches

In [36]:
delete_caches()

Deleting cache named "tenders and companies info" ...
All the caches have been deleted!
